In [ ]:
!pip install -U lancedb tantivy

In [ ]:
from pathlib import Path

from agno.agent import Agent
from agno.knowledge.embedder.openai import OpenAIEmbedder
from agno.knowledge.knowledge import Knowledge
from agno.models.openai import OpenAIChat
from agno.team.team import Team
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.vectordb.lancedb import LanceDb, SearchType

# Setup paths for knowledge storage
tmp_dir = Path("/content/tmp")
tmp_dir.mkdir(parents=True, exist_ok=True)

# Initialize knowledge base with vector database
agno_docs_knowledge = Knowledge(
    vector_db=LanceDb(
        uri=str(tmp_dir.joinpath("lancedb")),
        table_name="agno_docs",
        search_type=SearchType.hybrid,
        embedder=OpenAIEmbedder(id="text-embedding-3-small"),
    ),
)

# Add content to knowledge base
agno_docs_knowledge.add_content(url="https://docs.agno.com/llms-full.txt")

# Create web search agent for supplementary information
web_agent = Agent(
    name="Web Search Agent",
    role="Handle web search requests",
    model=Gemini(id="gemini-flash-lite-latest"),
    tools=[DuckDuckGoTools()],
    instructions=["Always include sources"],
)

# Create team with knowledge base integration
team_with_knowledge = Team(
    name="Team with Knowledge",
    members=[web_agent],
    model=Gemini(id="gemini-flash-lite-latest"),
    knowledge=agno_docs_knowledge,
    show_members_responses=True,
    markdown=True,
)

response = team_with_knowledge.run("Tell me about the Agno framework")


print(response.content)